# 뉴스 추천 시스템 점수화 EDA (news + toon 통합)

**의도적 행동 기반 점수 체계 설계**  
사용자의 의도적 행동 데이터만을 분석하여 News와 Toon 콘텐츠에 대한 각 이벤트의 상대적 중요도와 선호도를 객관적으로 도출하고, 이를 바탕으로 최적의 점수 체계를 제안합니다.

## 분석 목표
1. News + Toon 이벤트별 사용자 참여도와 선호도 정량화
2. 콘텐츠 유형별 사용자 행동 패턴 분석
3. 행동 시퀀스 패턴 분석을 통한 가중치 도출
4. 통계적 방법론 기반의 통합 점수 체계 설계

## 포함/제외 이벤트
### ✅ 포함 (의도적 행동)
- **News**: `news_click`, `news_view_end`, `news_bookmark`
- **Toon**: `toon_positive`, `toon_negative`, `toon_click`, `toon_expand_news`, `toon_impression`

### ❌ 제외 (불명확하거나 노이즈)
- `news_impression`: CTR 계산용, 단순 노출은 의도와 무관
- `news_view_start`: 시작만으로는 실제 관심도 불명확

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 설정
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False

# 스타일 설정
sns.set_style("whitegrid")
plt.style.use('default')

: 

## 1. 데이터 로드 및 기본 정보

In [ ]:
# CSV 파일 로드 (2025-09-25, 2025-09-26 데이터 합치기)
df_20250925 = pd.read_csv('data/events_20250925.csv')
df_20250926 = pd.read_csv('data/events_20250926.csv')

# 두 날짜 데이터 합치기
df_raw = pd.concat([df_20250925, df_20250926], ignore_index=True)

print(f"2025-09-25 데이터: {len(df_20250925):,}개")
print(f"2025-09-26 데이터: {len(df_20250926):,}개")
print(f"합친 전체 데이터: {len(df_raw):,}개")

# 추천 시스템에 적합한 이벤트만 필터링
# 제외: news_impression (단순 노출), news_view_start (시작만으로는 의도 불명확)
# 포함: 실제 사용자 의도가 명확한 행동들 + toon 관련 이벤트들
relevant_events = [
    'news_click', 'news_view_end', 'news_bookmark',  # 뉴스 관련 의도적 행동
    'toon_positive', 'toon_negative', 'toon_click',   # 툰 관련 명확한 반응
    'toon_expand_news', 'toon_impression'             # 툰 관련 추가 행동
]

df = df_raw[df_raw['event_name'].isin(relevant_events)].copy()

print(f"\n전체 데이터 크기: {df_raw.shape}")
print(f"필터링 후: {df.shape}")
print(f"제외된 이벤트 수: {len(df_raw) - len(df):,}개")
print(f"포함된 이벤트 타입: {sorted(df['event_name'].unique())}")
print(f"컬럼: {list(df.columns)}")
print(f"\n데이터 타입:")
print(df.dtypes)

# 기본 통계
print(f"\n=== 기본 통계 (의도적 행동만) ===")
print(f"총 이벤트 수: {len(df):,}")
print(f"유니크 사용자 수: {df['user_id'].nunique():,}")
print(f"유니크 뉴스 수: {df['news_id'].nunique():,}")
print(f"이벤트 종류: {df['event_name'].nunique()}개")

# 날짜별 데이터 분포 확인
df['date'] = pd.to_datetime(df['ts']).dt.date
print(f"\n=== 날짜별 분포 ===")
for date, count in df['date'].value_counts().sort_index().items():
    print(f"{date}: {count:,}개")

In [ ]:
# 데이터 미리보기
df.head(10)

## 2. 이벤트별 분포 분석

In [ ]:
# 이벤트별 빈도
event_counts = df['event_name'].value_counts()
print("=== 이벤트별 빈도 ===")
for event, count in event_counts.items():
    percentage = (count / len(df)) * 100
    print(f"{event}: {count:,} ({percentage:.1f}%)")

# 시각화
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
event_counts.plot(kind='bar', rot=45)
plt.title('Event Frequency')
plt.ylabel('Count')
plt.yscale('log')  # 로그 스케일로 차이를 명확히

plt.subplot(1, 2, 2)
event_counts.plot(kind='pie', autopct='%1.1f%%')
plt.title('Event Distribution')
plt.ylabel('')

plt.tight_layout()
plt.show()

## 3. 사용자 행동 패턴 분석

In [ ]:
# 사용자별 이벤트 수 분포
user_event_counts = df.groupby('user_id').size()

print(f"=== 사용자별 이벤트 수 통계 ===")
print(user_event_counts.describe())

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
user_event_counts.hist(bins=50, edgecolor='black', alpha=0.7)
plt.xlabel('Events per User')
plt.ylabel('Number of Users')
plt.title('Distribution of Events per User')
plt.yscale('log')

plt.subplot(1, 2, 2)
user_event_counts.plot(kind='box')
plt.ylabel('Events per User')
plt.title('Box Plot: Events per User')

plt.tight_layout()
plt.show()

# 활성 사용자 vs 비활성 사용자
high_activity_threshold = user_event_counts.quantile(0.8)
low_activity_threshold = user_event_counts.quantile(0.2)

print(f"\n=== 사용자 활동 세분화 ===")
print(f"고활성 사용자 (상위 20%): {high_activity_threshold:.0f}+ 이벤트")
print(f"저활성 사용자 (하위 20%): {low_activity_threshold:.0f}- 이벤트")

## 4. 이벤트별 사용자 참여도 분석

In [ ]:
# 이벤트별 사용자 참여 지표 계산
event_metrics = {}

for event in df['event_name'].unique():
    event_df = df[df['event_name'] == event]
    
    metrics = {
        'count': len(event_df),
        'unique_users': event_df['user_id'].nunique(),
        'unique_news': event_df['news_id'].nunique(),
        'user_penetration': event_df['user_id'].nunique() / df['user_id'].nunique(),
        'avg_per_user': len(event_df) / event_df['user_id'].nunique() if event_df['user_id'].nunique() > 0 else 0,
        'frequency_score': len(event_df) / len(df)  # 전체 대비 빈도
    }
    
    # 반복 행동 분석 (같은 사용자가 같은 뉴스에 대해 반복하는 정도)
    repeated_actions = event_df.groupby(['user_id', 'news_id']).size()
    metrics['repeat_rate'] = (repeated_actions > 1).mean()
    
    event_metrics[event] = metrics

# 결과 출력
print("=== 이벤트별 참여도 지표 ===")
metrics_df = pd.DataFrame(event_metrics).T
for event, row in metrics_df.iterrows():
    print(f"\n{event}:")
    print(f"  총 횟수: {row['count']:,}")
    print(f"  참여 사용자 수: {row['unique_users']:,}")
    print(f"  사용자 침투율: {row['user_penetration']*100:.1f}%")
    print(f"  사용자당 평균: {row['avg_per_user']:.1f}회")
    print(f"  반복 행동률: {row['repeat_rate']*100:.1f}%")

## 5. 체류시간과 스크롤 비율 분석

In [ ]:
# news_view_end 이벤트만 분석
view_end_df = df[df['event_name'] == 'news_view_end'].copy()

# 현재 점수 계산 함수 정의
def calculate_current_score(row):
    """현재 train_cf.py에서 사용하는 점수 계산 로직"""
    if row['event_name'] == 'news_view_end':
        dwell_ms = row.get('dwell_ms', 0) or 0
        scroll_pct = row.get('scroll_pct', 0) or 0
        if dwell_ms >= 15000 or scroll_pct >= 70:
            return 1.2  # 고품질 읽기
        else:
            return 0.6  # 일반 읽기
    return 0

if len(view_end_df) > 0:
    # 결측치 처리
    view_end_df['dwell_ms'] = view_end_df['dwell_ms'].fillna(0)
    view_end_df['scroll_pct'] = view_end_df['scroll_pct'].fillna(0)
    
    print(f"=== news_view_end 이벤트 분석 ({len(view_end_df):,}개) ===")
    print(f"체류시간 통계 (ms):")
    print(view_end_df['dwell_ms'].describe())
    print(f"\n스크롤 비율 통계 (%):")
    print(view_end_df['scroll_pct'].describe())
    
    # 현재 기준 (15초, 70%) 검증
    high_dwell = (view_end_df['dwell_ms'] >= 15000).sum()
    high_scroll = (view_end_df['scroll_pct'] >= 70).sum()
    high_engagement = ((view_end_df['dwell_ms'] >= 15000) | (view_end_df['scroll_pct'] >= 70)).sum()
    
    print(f"\n=== 현재 기준점 분석 ===")
    print(f"15초 이상 체류: {high_dwell:,} ({high_dwell/len(view_end_df)*100:.1f}%)")
    print(f"70% 이상 스크롤: {high_scroll:,} ({high_scroll/len(view_end_df)*100:.1f}%)")
    print(f"고품질 읽기 (둘 중 하나): {high_engagement:,} ({high_engagement/len(view_end_df)*100:.1f}%)")
    
    # 시각화
    plt.figure(figsize=(15, 10))
    
    # 체류시간 분포
    plt.subplot(2, 3, 1)
    view_end_df['dwell_ms'].hist(bins=50, edgecolor='black', alpha=0.7)
    plt.axvline(15000, color='red', linestyle='--', label='15s threshold')
    plt.xlabel('Dwell Time (ms)')
    plt.ylabel('Frequency')
    plt.title('Dwell Time Distribution')
    plt.legend()
    
    # 스크롤 비율 분포
    plt.subplot(2, 3, 2)
    view_end_df['scroll_pct'].hist(bins=50, edgecolor='black', alpha=0.7)
    plt.axvline(70, color='red', linestyle='--', label='70% threshold')
    plt.xlabel('Scroll Percentage')
    plt.ylabel('Frequency')
    plt.title('Scroll Percentage Distribution')
    plt.legend()
    
    # 체류시간 vs 스크롤 관계
    plt.subplot(2, 3, 3)
    plt.scatter(view_end_df['dwell_ms'], view_end_df['scroll_pct'], alpha=0.1, s=1)
    plt.axvline(15000, color='red', linestyle='--', alpha=0.5)
    plt.axhline(70, color='red', linestyle='--', alpha=0.5)
    plt.xlabel('Dwell Time (ms)')
    plt.ylabel('Scroll Percentage')
    plt.title('Dwell Time vs Scroll Percentage')
    
    # 체류시간 박스플롯 (로그스케일)
    plt.subplot(2, 3, 4)
    view_end_df[view_end_df['dwell_ms'] > 0]['dwell_ms'].plot(kind='box')
    plt.yscale('log')
    plt.ylabel('Dwell Time (ms, log scale)')
    plt.title('Dwell Time Box Plot')
    
    # 스크롤 비율 박스플롯
    plt.subplot(2, 3, 5)
    view_end_df['scroll_pct'].plot(kind='box')
    plt.ylabel('Scroll Percentage')
    plt.title('Scroll Percentage Box Plot')
    
    # 점수 분포 (현재 기준)
    plt.subplot(2, 3, 6)
    view_end_scored = view_end_df.apply(calculate_current_score, axis=1)
    view_end_scored.value_counts().sort_index().plot(kind='bar')
    plt.xlabel('Score')
    plt.ylabel('Count')
    plt.title('Current Scoring Distribution\n(news_view_end only)')
    plt.xticks(rotation=0)
    
    plt.tight_layout()
    plt.show()
else:
    print("news_view_end 이벤트가 없습니다.")

## 6. 북마크 행동 분석

In [ ]:
# 북마크 이벤트 분석
bookmark_df = df[df['event_name'] == 'news_bookmark'].copy()

if len(bookmark_df) > 0:
    print(f"=== 북마크 이벤트 분석 ({len(bookmark_df):,}개) ===")
    
    # action별 분포
    action_counts = bookmark_df['action'].value_counts()
    print(f"액션별 분포:")
    for action, count in action_counts.items():
        print(f"  {action}: {count:,} ({count/len(bookmark_df)*100:.1f}%)")
    
    # 북마크 추가 vs 제거 비율
    add_count = action_counts.get('add', 0)
    remove_count = action_counts.get('remove', 0)
    
    print(f"\n북마크 추가/제거 비율: {add_count:,} / {remove_count:,}")
    if remove_count > 0:
        retention_rate = (add_count - remove_count) / add_count * 100
        print(f"북마크 유지율: {retention_rate:.1f}%")
    
    # 사용자별 북마크 수
    user_bookmark_counts = bookmark_df[bookmark_df['action'] == 'add'].groupby('user_id').size()
    print(f"\n사용자별 북마크 수 통계:")
    print(user_bookmark_counts.describe())
    
    # 시각화
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 3, 1)
    action_counts.plot(kind='pie', autopct='%1.1f%%')
    plt.title('Bookmark Actions Distribution')
    plt.ylabel('')
    
    plt.subplot(1, 3, 2)
    if len(user_bookmark_counts) > 0:
        user_bookmark_counts.hist(bins=20, edgecolor='black', alpha=0.7)
        plt.xlabel('Bookmarks per User')
        plt.ylabel('Number of Users')
        plt.title('Bookmarks per User Distribution')
    
    plt.subplot(1, 3, 3)
    if len(user_bookmark_counts) > 0:
        user_bookmark_counts.plot(kind='box')
        plt.ylabel('Bookmarks per User')
        plt.title('Bookmarks per User Box Plot')
    
    plt.tight_layout()
    plt.show()
else:
    print("북마크 이벤트가 없습니다.")

## 7. 이벤트 시퀀스 분석

In [ ]:
# 시간순으로 정렬
df_sorted = df.sort_values(['user_id', 'ts']).copy()

# 사용자-뉴스별 이벤트 시퀀스 분석 (샘플)
sample_users = df_sorted['user_id'].unique()[:1000]  # 처음 1000명만
sample_df = df_sorted[df_sorted['user_id'].isin(sample_users)].copy()

# 클릭 후 읽기 완료 비율
click_to_view_sequences = []

for user_id in sample_users:
    user_events = sample_df[sample_df['user_id'] == user_id].copy()
    
    for news_id in user_events['news_id'].unique():
        news_events = user_events[user_events['news_id'] == news_id]['event_name'].tolist()
        
        # 클릭 후 view_end가 있는지 확인
        if 'news_click' in news_events:
            click_idx = news_events.index('news_click')
            has_view_end = 'news_view_end' in news_events[click_idx:]
            click_to_view_sequences.append({
                'user_id': user_id,
                'news_id': news_id,
                'clicked': True,
                'completed_reading': has_view_end
            })

if click_to_view_sequences:
    sequence_df = pd.DataFrame(click_to_view_sequences)
    completion_rate = sequence_df['completed_reading'].mean() * 100
    
    print(f"=== 클릭-읽기 완료 시퀀스 분석 (샘플) ===")
    print(f"클릭한 뉴스 수: {len(sequence_df):,}")
    print(f"읽기 완료율: {completion_rate:.1f}%")
    
    # 시각화
    plt.figure(figsize=(8, 6))
    sequence_df['completed_reading'].value_counts().plot(kind='pie', 
                                                        labels=['Not Completed', 'Completed'],
                                                        autopct='%1.1f%%')
    plt.title(f'Click to Read Completion Rate\n(Sample: {len(sequence_df)} clicks)')
    plt.ylabel('')
    plt.show()
    
    print(f"\n분석 의미: 클릭 이벤트의 {completion_rate:.1f}%만 실제 읽기로 이어짐")
    if completion_rate < 50:
        print("→ 클릭 점수(2.0)가 너무 높을 수 있음")
    else:
        print("→ 클릭이 실제 관심을 잘 나타냄")
else:
    print("클릭-읽기 시퀀스를 분석할 데이터가 충분하지 않습니다.")

## 8. 데이터 기반 점수 체계 도출

In [ ]:
print("=== 데이터 기반 점수 체계 도출 (news + toon 통합) ===")

# 1. 이벤트별 선호도 강도 계산 (여러 지표의 종합)
scoring_factors = {}

for event in event_metrics.keys():
    # 필터링된 이벤트만 처리
    if event not in relevant_events:
        continue
        
    metrics = event_metrics[event]
    
    # 정규화된 지표들 (0-1 스케일) - 관련 이벤트들만으로 재계산
    relevant_metrics = [event_metrics[k] for k in event_metrics.keys() if k in relevant_events]
    max_penetration = max([m['user_penetration'] for m in relevant_metrics])
    max_avg_per_user = max([m['avg_per_user'] for m in relevant_metrics])
    max_repeat_rate = max([m['repeat_rate'] for m in relevant_metrics])
    
    # 가중 점수 계산
    preference_strength = (
        metrics['user_penetration'] / max_penetration * 0.4 +  # 사용자 참여 범위
        (1 - metrics['frequency_score']) * 0.3 +  # 희소성 (낮은 빈도 = 높은 의도성)
        metrics['repeat_rate'] / max_repeat_rate * 0.3  # 반복 행동 (높은 의도성)
    )
    
    scoring_factors[event] = {
        'preference_strength': preference_strength,
        'rarity_bonus': 1 - metrics['frequency_score'],  # 희소성 보너스
        'engagement_depth': metrics['repeat_rate']  # 참여 깊이
    }

# 2. 행동 시퀀스 기반 가중치 조정 (news + toon 통합)
sequence_weights = {
    # News 이벤트
    'news_bookmark': 1.0,      # 명확한 선호 표현 (최고)
    'news_view_end': 0.8,      # 실제 소비 완료
    'news_click': 0.6,         # 초기 관심
    
    # Toon 이벤트 
    'toon_positive': 0.7,      # 긍정 반응 (명확한 선호)
    'toon_negative': 0.3,      # 부정 반응 (낮은 점수지만 신호로 활용)
    'toon_click': 0.5,         # 툰 클릭 (news_click보다 약간 낮음)
    'toon_expand_news': 0.6,   # 뉴스 확장 (관심 표현)
    'toon_impression': 0.2,    # 툰 노출 (news_impression보다는 의미있음)
}

# 3. 최종 점수 계산 (news + toon 통합)
final_scores = {}
max_base_score = 5.0  # 최고점 기준

for event in scoring_factors.keys():
    if event in sequence_weights:
        # 데이터 기반 선호도 + 행동 시퀀스 가중치
        combined_score = (
            scoring_factors[event]['preference_strength'] * 0.6 +
            sequence_weights[event] * 0.4
        ) * max_base_score
        
        final_scores[event] = round(combined_score, 1)

# 4. 이벤트 카테고리별 정리
news_events = {k: v for k, v in final_scores.items() if k.startswith('news_')}
toon_events = {k: v for k, v in final_scores.items() if k.startswith('toon_')}

print(f"\n=== 제안하는 점수 체계 (news + toon 통합) ===")

print(f"\n📰 News 이벤트 점수:")
for event, score in sorted(news_events.items(), key=lambda x: x[1], reverse=True):
    pref = scoring_factors[event]['preference_strength']
    print(f"  {event}: {score} (선호도 강도: {pref:.3f})")

print(f"\n🎨 Toon 이벤트 점수:")
for event, score in sorted(toon_events.items(), key=lambda x: x[1], reverse=True):
    pref = scoring_factors[event]['preference_strength']
    print(f"  {event}: {score} (선호도 강도: {pref:.3f})")

# 5. 특별 처리: news_view_end 품질 구분
if 'view_end_df' in locals() and len(view_end_df) > 0:
    dwell_75 = view_end_df[view_end_df['dwell_ms'] > 0]['dwell_ms'].quantile(0.75)
    scroll_75 = view_end_df[view_end_df['scroll_pct'] > 0]['scroll_pct'].quantile(0.75)
    
    high_quality_score = final_scores.get('news_view_end', 2.0) * 1.2
    low_quality_score = final_scores.get('news_view_end', 2.0) * 0.7
    
    print(f"\n📖 읽기 품질별 점수 (news_view_end):")
    print(f"  고품질 읽기: {high_quality_score:.1f} (체류 {dwell_75/1000:.1f}초+ 또는 스크롤 {scroll_75:.0f}%+)")
    print(f"  일반 읽기: {low_quality_score:.1f} (그 외)")

# 6. 점수 체계의 논리적 근거
print(f"\n=== 점수 체계 설계 근거 ===")
print(f"1. 사용자 참여도 (40%): 얼마나 많은 사용자가 해당 행동을 하는가")
print(f"2. 희소성 (30%): 자주 일어나지 않을수록 의도성이 높음")
print(f"3. 반복성 (30%): 같은 행동을 반복할수록 강한 선호")
print(f"4. 행동 시퀀스 가중치: 사용자 여정에서의 행동 단계별 중요도")
print(f"5. News vs Toon: 컨텐츠 유형별 사용자 행동 패턴 차이 반영")

print(f"\n=== 제외된 이벤트와 이유 ===")
print(f"• news_impression: 단순 노출, 사용자 의도와 무관")
print(f"• news_view_start: 시작만으로는 실제 관심 불명확")
print(f"• 기타 노출 관련 이벤트: 추천 점수에 노이즈가 될 가능성")

print(f"\n=== 현재 모델과의 비교 ===")
print(f"현재 train_cf.py 점수:")
print(f"  • news_bookmark (add): 3.0")
print(f"  • news_click: 2.0")
print(f"  • news_view_end (고품질): 1.2")
print(f"  • news_view_end (일반): 0.6")
print(f"  • toon_expand_news: 1.0")
print(f"  • toon_positive: 0.8")
print(f"  • 기타 toon 이벤트: 미포함")

print(f"\n=== 검증 및 다음 단계 ===")
print(f"1. toon 이벤트를 포함한 모델 재학습")
print(f"2. news vs toon 콘텐츠별 추천 성능 비교")
print(f"3. 긍정/부정 반응 이벤트의 효과 검증")
print(f"4. 사용자 콘텐츠 선호도 다양성 개선 측정")

## 9. 요약 및 결론

In [ ]:
print("=" * 60)
print("📊 데이터 기반 점수 설계 리포트 (news + toon 통합)")
print("=" * 60)

print(f"\n📈 데이터 개요:")
print(f"  • 전체 이벤트: {len(df_raw):,}개")
print(f"  • 분석 대상 이벤트: {len(df):,}개")
print(f"  • 제외된 이벤트: {len(df_raw) - len(df):,}개")
print(f"  • 사용자 수: {df['user_id'].nunique():,}명")
print(f"  • 뉴스 수: {df['news_id'].nunique():,}개")
print(f"  • 이벤트 타입: {len(event_counts)}개")

print(f"\n🎯 핵심 인사이트:")

# 이벤트 분포
top_event = event_counts.index[0]
top_event_pct = (event_counts.iloc[0] / len(df)) * 100
print(f"  • 최빈 이벤트: {top_event} ({top_event_pct:.1f}%)")

# 사용자 활동
avg_events_per_user = user_event_counts.mean()
print(f"  • 사용자당 평균 이벤트: {avg_events_per_user:.1f}개")

# News vs Toon 분포
news_events_count = len(df[df['event_name'].str.startswith('news_')])
toon_events_count = len(df[df['event_name'].str.startswith('toon_')])
news_pct = (news_events_count / len(df)) * 100
toon_pct = (toon_events_count / len(df)) * 100

print(f"  • News 이벤트: {news_events_count:,}개 ({news_pct:.1f}%)")
print(f"  • Toon 이벤트: {toon_events_count:,}개 ({toon_pct:.1f}%)")

# 선호도 강도가 가장 높은 이벤트
if 'scoring_factors' in locals():
    top_preference = max(scoring_factors.items(), key=lambda x: x[1]['preference_strength'])
    print(f"  • 최고 선호도 이벤트: {top_preference[0]} (강도: {top_preference[1]['preference_strength']:.3f})")

# 희소성이 가장 높은 이벤트 (의도성 지표)
if 'scoring_factors' in locals():
    top_rarity = max(scoring_factors.items(), key=lambda x: x[1]['rarity_bonus'])
    print(f"  • 최고 의도성 이벤트: {top_rarity[0]} (희소성: {top_rarity[1]['rarity_bonus']:.3f})")

print(f"\n💡 설계 원칙:")
print(f"  1. 의도적 행동만 측정: 단순 노출과 불확실한 시작 이벤트 제외")
print(f"  2. 다차원 평가: 참여도, 희소성, 반복성 종합")
print(f"  3. 콘텐츠 유형별 차별화: News와 Toon의 사용자 행동 패턴 반영")
print(f"  4. 품질 차별화: 같은 이벤트 내 품질 구분")

print(f"\n🚫 제외된 이벤트와 이유:")
print(f"  • news_impression: CTR 계산용, 단순 노출은 의도와 무관")
print(f"  • news_view_start: 시작만으로는 실제 관심도 불명확")
print(f"  • 기타 노출 관련: 추천 시스템에 노이즈로 작용 가능")

print(f"\n🎨 Toon 이벤트 추가 이유:")
print(f"  • 사용자 콘텐츠 선호도 다양성 확보")
print(f"  • 툰 형태의 뉴스 소비 패턴 반영")
print(f"  • 긍정/부정 반응을 통한 더 정교한 선호도 측정")
print(f"  • 뉴스 확장 행동을 통한 관심도 파악")

print(f"\n🔄 실행 방안:")
print(f"  • toon 이벤트를 포함한 점수로 추천 모델 재학습")
print(f"  • news vs toon 콘텐츠별 추천 성능 비교")
print(f"  • 기존 vs 신규 점수 체계 A/B 테스트")
print(f"  • 사용자 콘텐츠 다양성 소비 패턴 모니터링")

print("\n" + "=" * 60)
print(f"분석 완료: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 60)